<a href="https://colab.research.google.com/github/scenaristeur/igora/blob/main/notebooks/llama_cpp_python_fr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation du moteur d'inférence llama-cpp-python version CPU
https://llama-cpp-python.readthedocs.io/en/latest/

In [ ]:
!pip install llama-cpp-python[server] --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cpu

### Téléchargement d'un modèle de langage
on va utiliser le modèle "dolphin" au format GGUF (https://huggingface.co/TheBloke/dolphin-2.2.1-mistral-7B-GGUF/) en le téléchargeant sur https://huggingface.co/TheBloke dans le dossier ./models

In [ ]:
!mkdir models
!wget -O ./models/dolphin-2.2.1-mistral-7b.Q2_K.gguf https://huggingface.co/TheBloke/dolphin-2.2.1-mistral-7B-GGUF/resolve/main/dolphin-2.2.1-mistral-7b.Q2_K.gguf?download=true

### On peut maintenant poser une question au modèle de langage

In [ ]:
from llama_cpp import Llama
llm = Llama(
      model_path="./models/dolphin-2.2.1-mistral-7b.Q2_K.gguf",
      chat_format="chatml"
      # n_gpu_layers=-1, # Uncomment to use GPU acceleration
      # seed=1337, # Uncomment to set a specific seed
      # n_ctx=2048, # Uncomment to increase the context window
)
output = llm(
      "Q: Name the planets in the solar system? A: ", # Prompt
      max_tokens=150, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
      echo=False, # Echo the prompt back in the output
      #stream=True
) # Generate a completion, can also call create_completion

print("############ REPONSE ############")
print(output)
reponse = output['choices'][0]["text"]
print("\nRéponse : ", reponse)

In [ ]:
output = llm(
      "Je vais vous raconter l'histoire de Toto : ", # Prompt
      max_tokens=150, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
      echo=False, # Echo the prompt back in the output
      #stream=True
) # Generate a completion, can also call create_completion

print("############ REPONSE ############")
print(output)
reponse = output['choices'][0]["text"]
print("\nRéponse : ", reponse)

In [ ]:
llm = Llama(model_path="./models/dolphin-2.2.1-mistral-7b.Q2_K.gguf", chat_format="chatml-function-calling")
output = llm.create_chat_completion(
      messages = [
        {
          "role": "system",
          "content": "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. The assistant calls functions with appropriate input when necessary"

        },
        {
          "role": "user",
          "content": "Extract Jason is 25 years old"
        }
      ],
      tools=[{
        "type": "function",
        "function": {
          "name": "UserDetail",
          "parameters": {
            "type": "object",
            "title": "UserDetail",
            "properties": {
              "name": {
                "title": "Name",
                "type": "string"
              },
              "age": {
                "title": "Age",
                "type": "integer"
              }
            },
            "required": [ "name", "age" ]
          }
        }
      }],
      tool_choice={
        "type": "function",
        "function": {
          "name": "UserDetail"
        }
      }
)

print("############ REPONSE ############")
print(output)
reponse = output['choices'][0]["message"]
print("\nRéponse : ", reponse)
json_data = reponse["function_call"]
print( "JSON data : ",json_data)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ./models/dolphin-2.2.1-mistral-7b.Q2_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = ehartford_dolphin-2.2.1-mistral-7b
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:           

In [ ]:
from llama_cpp.llama_chat_format import MoondreamChatHandler

chat_handler = MoondreamChatHandler.from_pretrained(
  repo_id="vikhyatk/moondream2",
  filename="*mmproj*",
)

llm = Llama.from_pretrained(
  repo_id="vikhyatk/moondream2",
  filename="*text-model*",
  chat_handler=chat_handler,
  n_ctx=2048, # n_ctx should be increased to accommodate the image embedding
)

response = llm.create_chat_completion(
    messages = [
        {
            "role": "user",
            "content": [
                {"type" : "text", "text": "What's in this image?"},
                {"type": "image_url", "image_url": {"url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg" } }

            ]
        }
    ]
)
print(response["choices"][0]["text"])